In [3]:
import numpy as np
from fractions import Fraction
from math import gcd
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram

In [14]:
def qpe_amod15(a):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q), 
                 [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) # Do inverse-QFT
    qc.measure(range(n_count), range(n_count))
    
    aer_sim = Aer.get_backend('aer_simulator')
    # Using Aer's unitary simulator for c_amod15
    # to mitigate the 'unknown instruction' error
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(qobj, memory=True).result()
    readings = result.get_memory()
    print("Register Reading: " + readings[0])
    print("Corresponding Value: ", int(readings[0],2))
    return int(readings[0],2)


In [15]:
def c_amod15(a, power):
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

In [16]:
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

In [17]:
# Calling the function to factorize 15
np.random.seed = 2
a = np.random.randint(2,15)
print("Chosen a is", a)
factor_found = False
attempt = 0
while not factor_found:
    attempt += 1
    print(f"Attempt {attempt}...")
    measured = qpe_amod15(a)
    fraction = Fraction(measured/2**8).limit_denominator(15)
    # continued fraction expansion
    s, d = fraction.numerator, fraction.denominator
    print("Numerator of fraction: ", s)
    print("Denominator of fraction: ", d)
    # we guess r based on the periodicity
    guesses = [gcd(a**(d//r)-1,15), gcd(a**(d//r)+1,15)]
    print("Guessed Factors: ", guesses)
    for guess in guesses:
        if guess not in [1, 15] and (15 % guess) == 0:
            print("Factors found:", guess, 15//guess)
            factor_found = True

if not factor_found:
    print("Failed to find factors within a reasonable number of attempts.")

Chosen a is 3
Attempt 1...


C:\Users\91952\AppData\Local\Temp\ipykernel_10596\2712324593.py:18: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = aer_sim.run(qobj, memory=True).result()


Register Reading: 11000000
Corresponding Value:  192
Numerator of fraction:  3
Denominator of fraction:  4


NameError: name 'r' is not defined

In [18]:
import numpy as np
from fractions import Fraction
from math import gcd
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram

# Define the controlled modular multiplication gate
def c_amod15(a, power):
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

# Define the quantum inverse Fourier transform (QFT) gate
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

# Implement Shor's algorithm
def shors_algorithm(N):
    attempts = 0
    while True:
        attempts += 1
        print(f"Attempt {attempts}...")
        
        a = np.random.randint(2, N)
        if gcd(a, N) > 1:
            print("Guessed a:", a)
            return gcd(a, N)
        
        measured = qpe_amod15(a, N)
        fraction = Fraction(measured/2**8).limit_denominator(N)
        s, d = fraction.numerator, fraction.denominator
        
        guesses = [gcd(a**(d//r)-1, N), gcd(a**(d//r)+1, N)]
        print("Guesses for factors:", guesses)
        
        for guess in guesses:
            if guess != 1 and guess != N and N % guess == 0:
                print("Factors found:", guess, N//guess)
                return guess, N//guess

N = 15  # Number to factorize
factor1, factor2 = shors_algorithm(N)
print(f"Factors of {N}: {factor1}, {factor2}")


Attempt 1...
Guessed a: 12


TypeError: cannot unpack non-iterable int object

In [19]:
import numpy as np
from math import gcd
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram

def qpe_amod15(a, N):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q, N), 
                 [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) # Do inverse-QFT
    qc.measure(range(n_count), range(n_count))
    
    aer_sim = Aer.get_backend('aer_simulator')
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(qobj, memory=True).result()
    readings = result.get_memory()
    return int(readings[0],2)

def c_amod15(a, power, N):
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

def shors_algorithm(N):
    attempts = 0
    while True:
        attempts += 1
        a = np.random.randint(2, N)
        if gcd(a, N) > 1:
            print("Guessed a:", a)
            return gcd(a, N)
        
        measured = qpe_amod15(a, N)
        fraction = Fraction(measured/2**8).limit_denominator(N)
        s, d = fraction.numerator, fraction.denominator
        
        guesses = [gcd(a**(d//r)-1, N), gcd(a**(d//r)+1, N)]
        for guess in guesses:
            if guess != 1 and guess != N and N % guess == 0:
                print("Factors found:", guess, N//guess)
                return guess, N//guess

N = 15  # Number to factorize
factor1, factor2 = shors_algorithm(N)
print(f"Factors of {N}: {factor1}, {factor2}")


C:\Users\91952\AppData\Local\Temp\ipykernel_10596\1304953587.py:21: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = aer_sim.run(qobj, memory=True).result()


NameError: name 'r' is not defined

In [20]:
import numpy as np
from fractions import Fraction
from math import gcd
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram

def qpe_amod15(a, N):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q, N), 
                 [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) # Do inverse-QFT
    qc.measure(range(n_count), range(n_count))
    
    aer_sim = Aer.get_backend('aer_simulator')
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(qobj, memory=True).result()
    readings = result.get_memory()
    return int(readings[0],2)

def c_amod15(a, power, N):
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

def shors_algorithm(N):
    attempts = 0
    while True:
        attempts += 1
        a = np.random.randint(2, N)
        if gcd(a, N) > 1:
            print("Guessed a:", a)
            return gcd(a, N)
        
        measured = qpe_amod15(a, N)
        fraction = Fraction(measured/2**8).limit_denominator(N)
        s, d = fraction.numerator, fraction.denominator
        
        guesses = [gcd(a**(d//guess)-1, N) for guess in range(1, d+1)]
        for guess in guesses:
            if guess != 1 and guess != N and N % guess == 0:
                print("Factors found:", guess, N//guess)
                return guess, N//guess

N = 15  # Number to factorize
factor1, factor2 = shors_algorithm(N)
print(f"Factors of {N}: {factor1}, {factor2}")


Guessed a: 3


TypeError: cannot unpack non-iterable int object

In [21]:
import numpy as np
from math import gcd
from qiskit import QuantumCircuit, Aer, transpile, assemble

# Define the controlled modular multiplication gate
def c_amod15(a, power, N):
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.swap(2, 3)
        U.swap(1, 2)
        U.swap(0, 1)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod {N}"
    c_U = U.control()
    return c_U

# Implement Shor's algorithm
def shors_algorithm(N):
    attempts = 0
    while True:
        attempts += 1
        a = np.random.randint(2, N)
        if gcd(a, N) > 1:
            print("Guessed a:", a)
            return gcd(a, N)
        
        measured = qpe_amod15(a, N)
        s, d = continued_fractions(measured / 2**8, N)
        
        guesses = [gcd(a**(d//guess)-1, N) for guess in range(1, d+1)]
        for guess in guesses:
            if guess != 1 and guess != N and N % guess == 0:
                print("Factors found:", guess, N//guess)
                return guess, N//guess

def qpe_amod15(a, N):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q, N), 
                 [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) # Do inverse-QFT
    qc.measure(range(n_count), range(n_count))
    
    aer_sim = Aer.get_backend('aer_simulator')
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(qobj, memory=True).result()
    readings = result.get_memory()
    return int(readings[0],2)

def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

def continued_fractions(x, N):
    f = Fraction(x).limit_denominator(N)
    return f.numerator, f.denominator

N = 15  # Number to factorize
factor1, factor2 = shors_algorithm(N)
print(f"Factors of {N}: {factor1}, {factor2}")


Guessed a: 5


TypeError: cannot unpack non-iterable int object